# Logging RGB, D, Force Data

The goal of this script is to log the RGB Camera, Depth Camera, and Force Data at the same sampling rate and time via the serial port. 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pyrealsense2 as rs
import cv2
import serial

# Setup:
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)


# Create an object to read  
# from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera 
# is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 
    
# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 



# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
colorizer = rs.colorizer()
mag_deci = 4
decimation = rs.decimation_filter(mag_deci)
threshold = rs.threshold_filter(0.15, 1.6)

n_trial = 18
depthwriter = cv2.VideoWriter('fcss_leo_depth_test'+str(n_trial)+'.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         30, (int(frame_width/mag_deci), int(frame_height/mag_deci)), 1) 

colorwriter = cv2.VideoWriter('fcss_leo_color_test'+str(n_trial)+'.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         30, (int(frame_width/mag_deci), int(frame_height/mag_deci)), 1) 

print("type anything to start")
start_test = input()


addr = "COM4" ## serial port to read data from
baud = 9600 ## baud rate for instrument

ser = serial.Serial(
    port = addr,\
    baudrate = baud)

fcss_filename = "fcss_leo_load_test"+str(n_trial)+".csv"
datafile=open(fcss_filename, 'a')




# Start streaming
pipe.start(cfg)


try:
    while True:
        try:
            ser_bytes = ser.readline()
            decoded_bytes = (ser_bytes[0:len(ser_bytes)-2].decode("utf-8").split(","))

            print(decoded_bytes)

            write_line = ",".join(decoded_bytes) + str("\n")
            datafile.write(write_line)
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipe.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
            
            if not depth_frame or not color_frame:
                continue
            
            # Decimation 
            depth_frame = decimation.process(depth_frame)

            # Thresholding
            depth_frame = threshold.process(depth_frame)
        
            
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
#             depth_image = np.asanyarray(depth_frame.get_data())
#             depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            depth_colormap = np.asanyarray(colorizer.colorize(depth_frame).get_data())
            color_image = np.asanyarray(color_frame.get_data())
        
            key = cv2.waitKey(1)
            
            depth_save = cv2.resize(depth_colormap, (int(frame_width/mag_deci), int(frame_height/mag_deci)))
            
            color_save = cv2.resize(color_image, (int(frame_width/mag_deci), int(frame_height/mag_deci)))
            
            depthwriter.write(depth_save)
            colorwriter.write(color_save)
            cv2.imshow('Depth', depth_save)
            cv2.imshow('RGB', color_save)

            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                video.release() 
                depthwriter.release()
                colorwriter.release()
                break 
                
        except:
            pass
            break

finally:
    ser.close()
    datafile.close()
    pipe.stop()
    cv2.destroyAllWindows()
    video.release() 
    colorwriter.release()
    depthwriter.release()

# Logging Force Data Only

In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import serial
import time


print("type anything to start")
start_test = input()

n_trial = 22
addr = "COM4" ## serial port to read data from
baud = 115200 ## baud rate for instrument

ser = serial.Serial(
    port = addr,\
    baudrate = baud)

fcss_filename = "fcss_leo_load_test"+str(n_trial)+".csv"
datafile=open(fcss_filename, 'a')

previous_time = 0
time_sample = 0.016667

try: 
    while True:
        
        try:
#             current_time = time.time()
            
#             if (current_time - previous_time) >= time_sample:
#                 print(current_time - previous_time)
#             previous_time = current_time
            ser_bytes = ser.readline()
            decoded_bytes = (ser_bytes[0:len(ser_bytes)-2].decode("utf-8").split(","))



            write_line = ",".join(decoded_bytes) + str("\n")
            print(write_line)
            datafile.write(write_line)


        except:
            pass
            break
            
        

finally:
    ser.close()
    datafile.close()
    

type anything to start
s
1023,-9.78,-2.40,-2.88,1.89,2.43,0.11

1023,-9.75,-2.40,-2.84,1.89,2.44,0.13

1023,-9.76,-2.41,-2.86,1.88,2.43,0.13

1023,-9.76,-2.40,-2.86,1.90,2.44,0.11

1023,-9.76,-2.39,-2.87,1.89,2.45,0.11

1023,-9.76,-2.41,-2.87,1.89,2.43,0.13

1023,-9.76,-2.39,-2.86,1.89,2.44,0.12

1023,-9.76,-2.39,-2.87,1.88,2.44,0.12

1023,-9.76,-2.38,-2.86,1.90,2.43,0.11

1023,-9.75,-2.42,-2.88,1.88,2.45,0.13

1023,-9.76,-2.40,-2.86,1.90,2.44,0.12

1023,-9.79,-2.40,-2.87,1.89,2.43,0.11

1023,-9.76,-2.39,-2.87,1.88,2.44,0.12

1023,-9.74,-2.41,-2.87,1.89,2.42,0.13

1023,-9.75,-2.39,-2.85,1.90,2.46,0.12

1023,-9.77,-2.40,-2.86,1.89,2.45,0.11

1023,-9.76,-2.42,-2.86,1.89,2.44,0.12

1023,-9.76,-2.41,-2.86,1.87,2.43,0.14

1023,-9.76,-2.41,-2.86,1.89,2.43,0.11

1023,-9.78,-2.40,-2.86,1.88,2.45,0.11

1023,-9.76,-2.41,-2.86,1.90,2.45,0.12

1023,-9.76,-2.41,-2.87,1.89,2.45,0.13

1023,-9.77,-2.39,-2.86,1.89,2.44,0.12

1023,-9.78,-2.40,-2.86,1.89,2.43,0.11

1023,-9.76,-2.39,-2.85,1.88,2.44,0.13




1023,-9.76,-2.41,-2.85,1.89,2.44,0.12

1023,-9.77,-2.40,-2.86,1.88,2.43,0.13

1023,-9.76,-2.40,-2.86,1.89,2.43,0.11

1023,-9.76,-2.40,-2.87,1.88,2.44,0.12

1023,-9.76,-2.40,-2.86,1.89,2.43,0.12

1023,-9.75,-2.41,-2.87,1.89,2.44,0.12

1023,-9.78,-2.39,-2.87,1.89,2.44,0.12

1023,-9.76,-2.39,-2.86,1.90,2.42,0.12

1023,-9.76,-2.40,-2.86,1.87,2.44,0.13

1023,-9.77,-2.40,-2.86,1.89,2.43,0.12

1023,-9.77,-2.40,-2.87,1.88,2.44,0.12

1023,-9.77,-2.39,-2.85,1.89,2.43,0.12

1023,-9.76,-2.40,-2.86,1.89,2.42,0.12

1023,-9.75,-2.39,-2.87,1.89,2.43,0.12

1023,-9.78,-2.41,-2.87,1.88,2.43,0.12

1023,-9.75,-2.40,-2.86,1.89,2.45,0.13

1023,-9.78,-2.41,-2.86,1.89,2.45,0.13

1023,-9.77,-2.41,-2.88,1.88,2.43,0.12

1023,-9.76,-2.40,-2.87,1.88,2.43,0.12

1023,-9.76,-2.41,-2.86,1.89,2.42,0.12

1023,-9.76,-2.40,-2.87,1.89,2.45,0.12

1023,-9.77,-2.39,-2.85,1.89,2.44,0.12

1023,-9.78,-2.41,-2.87,1.90,2.44,0.12

1023,-9.75,-2.41,-2.86,1.89,2.44,0.13

1023,-9.77,-2.39,-2.86,1.89,2.42,0.13

1023,-9.74,-2.40,-2.87,1

1023,-9.76,-2.40,-2.87,1.88,2.43,0.11

1023,-9.76,-2.41,-2.86,1.89,2.42,0.12

1023,-9.75,-2.41,-2.88,1.88,2.44,0.13

1023,-9.77,-2.40,-2.86,1.88,2.45,0.12

1023,-9.77,-2.40,-2.87,1.89,2.44,0.11

1023,-9.77,-2.40,-2.86,1.88,2.44,0.12

1023,-9.76,-2.41,-2.86,1.88,2.43,0.13

1023,-9.75,-2.40,-2.86,1.88,2.44,0.12

1023,-9.77,-2.39,-2.87,1.90,2.44,0.11

1023,-9.76,-2.42,-2.87,1.89,2.43,0.12

1023,-9.78,-2.41,-2.85,1.89,2.45,0.13

1023,-9.76,-2.41,-2.85,1.89,2.44,0.11

1023,-9.77,-2.39,-2.87,1.89,2.45,0.11

1023,-9.76,-2.41,-2.86,1.89,2.44,0.13

1023,-9.77,-2.40,-2.86,1.89,2.43,0.12

1023,-9.76,-2.38,-2.87,1.88,2.43,0.11

1023,-9.77,-2.41,-2.86,1.90,2.42,0.11

1023,-9.77,-2.41,-2.86,1.87,2.45,0.12

1023,-9.76,-2.40,-2.85,1.89,2.43,0.12

1023,-9.77,-2.40,-2.86,1.89,2.44,0.11

1023,-9.76,-2.40,-2.86,1.90,2.45,0.12

1023,-9.75,-2.40,-2.87,1.88,2.43,0.12

1023,-9.77,-2.40,-2.86,1.88,2.44,0.12

1023,-9.78,-2.41,-2.86,1.89,2.45,0.11

1023,-9.77,-2.41,-2.87,1.90,2.45,0.12

1023,-9.76,-2.41,-2.87,1.

1023,-9.77,-2.40,-2.87,1.89,2.43,0.11

1023,-9.75,-2.40,-2.86,1.89,2.42,0.12

1023,-9.77,-2.41,-2.85,1.88,2.43,0.13

1023,-9.76,-2.40,-2.86,1.90,2.45,0.12

1023,-9.77,-2.41,-2.87,1.89,2.45,0.11

1023,-9.75,-2.40,-2.87,1.89,2.42,0.12

1023,-9.77,-2.41,-2.86,1.88,2.44,0.12

1023,-9.77,-2.40,-2.86,1.89,2.43,0.11

1023,-9.77,-2.41,-2.87,1.89,2.45,0.11

1023,-9.77,-2.40,-2.86,1.88,2.44,0.12

1023,-9.76,-2.40,-2.86,1.89,2.44,0.13

1023,-9.76,-2.40,-2.87,1.90,2.43,0.11

1023,-9.77,-2.39,-2.86,1.89,2.43,0.11

1023,-9.76,-2.41,-2.86,1.89,2.45,0.13

1023,-9.77,-2.40,-2.87,1.89,2.44,0.12

1023,-9.77,-2.40,-2.86,1.88,2.45,0.11

1023,-9.76,-2.39,-2.87,1.88,2.44,0.11

1023,-9.75,-2.40,-2.87,1.88,2.44,0.12

1023,-9.75,-2.40,-2.87,1.89,2.44,0.12

1023,-9.78,-2.40,-2.86,1.90,2.44,0.11

1023,-9.76,-2.41,-2.86,1.89,2.43,0.12

1023,-9.78,-2.41,-2.86,1.88,2.44,0.13

1023,-9.75,-2.41,-2.87,1.89,2.42,0.12

1023,-9.76,-2.40,-2.85,1.89,2.43,0.11

1023,-9.76,-2.41,-2.87,1.89,2.44,0.12

1023,-9.77,-2.40,-2.87,1.

1023,-9.77,-2.41,-2.87,1.90,2.43,0.11

1023,-9.75,-2.40,-2.87,1.89,2.43,0.12

1023,-9.76,-2.42,-2.87,1.88,2.43,0.13

1023,-9.76,-2.41,-2.86,1.90,2.44,0.11

1023,-9.77,-2.41,-2.86,1.90,2.44,0.12

1023,-9.78,-2.41,-2.87,1.90,2.43,0.13

1023,-9.76,-2.40,-2.87,1.88,2.43,0.13

1023,-9.77,-2.41,-2.86,1.89,2.43,0.11

1023,-9.76,-2.39,-2.88,1.89,2.44,0.11

1023,-9.76,-2.41,-2.87,1.89,2.46,0.13

1023,-9.76,-2.41,-2.87,1.89,2.44,0.12

1023,-9.75,-2.40,-2.86,1.88,2.44,0.12

1023,-9.78,-2.40,-2.86,1.88,2.43,0.12

1023,-9.76,-2.39,-2.86,1.88,2.43,0.12

1023,-9.76,-2.41,-2.86,1.89,2.46,0.12

1023,-9.76,-2.41,-2.87,1.89,2.44,0.11

1023,-9.76,-2.40,-2.86,1.90,2.43,0.12

1023,-9.78,-2.40,-2.86,1.90,2.43,0.13

1023,-9.77,-2.40,-2.87,1.89,2.43,0.11

1023,-9.76,-2.40,-2.86,1.90,2.44,0.12

1023,-9.77,-2.39,-2.87,1.89,2.42,0.11

1023,-9.75,-2.41,-2.87,1.88,2.46,0.12

1023,-9.77,-2.40,-2.87,1.88,2.43,0.11

1023,-9.76,-2.42,-2.86,1.89,2.43,0.11

1023,-9.75,-2.40,-2.86,1.87,2.43,0.13

1023,-9.77,-2.41,-2.87,1.

...

1023,-56.85,-12.51,-10.73,10.03,11.94,0.32

1023,-56.77,-12.61,-10.78,10.04,11.93,0.33

1023,-56.67,-12.66,-10.84,10.03,11.95,0.35

1023,-56.61,-12.70,-10.84,10.05,11.93,0.35

1023,-56.56,-12.73,-10.86,10.04,11.89,0.36

1023,-56.53,-12.76,-10.86,10.03,11.89,0.36

1023,-56.53,-12.78,-10.85,9.98,11.88,0.35

1023,-56.52,-12.82,-10.86,9.97,11.89,0.35

1023,-56.51,-12.85,-10.87,9.99,11.89,0.35

1023,-56.55,-12.90,-10.85,10.00,11.91,0.36

1023,-56.57,-12.93,-10.85,10.03,11.91,0.37

1023,-56.53,-12.96,-10.83,10.03,11.91,0.37

1023,-56.51,-12.97,-10.80,10.03,11.90,0.38

1023,-56.43,-13.00,-10.78,10.04,11.88,0.38

1023,-56.40,-12.99,-10.78,10.02,11.87,0.39

1023,-56.42,-13.02,-10.77,10.04,11.88,0.40

1023,-56.42,-13.01,-10.77,10.06,11.88,0.40

1023,-56.45,-13.06,-10.76,10.07,11.89,0.40

1023,-56.45,-13.07,-10.76,10.06,11.90,0.39

1023,-56.47,-13.08,-10.75,10.06,11.89,0.40

1023,-56.47,-13.09,-10.76,10.05,11.89,0.40

1023,-56.45,-13.09,-10.75,10.05,11.88,0.41

1023,-56.40,-13.10,-10.77,10.03,11.

1023,-50.68,2.01,-28.20,4.44,20.37,3.62

1023,-50.31,2.62,-30.00,4.48,20.47,3.15

1023,-49.78,3.34,-32.17,4.32,21.01,2.85

1023,-48.76,4.04,-34.24,4.06,21.47,2.65

1023,-46.79,4.21,-35.09,4.41,21.27,1.68

1023,-43.26,4.63,-35.50,4.39,20.35,0.98

1023,-38.34,4.04,-35.24,4.20,18.68,0.04

1023,-32.77,2.39,-33.27,3.92,16.10,-0.22

1023,-27.16,0.43,-30.45,3.58,13.98,-0.16

1023,-21.24,-1.34,-26.98,3.88,11.41,-0.26

1023,-16.16,-4.77,-24.24,3.82,9.23,-0.23

1023,-10.71,-7.50,-21.66,2.77,8.54,0.19

1023,-7.78,-9.33,-19.34,3.21,7.44,0.19

1023,-6.58,-9.91,-17.65,4.19,6.45,-0.19

1023,-5.40,-9.90,-16.26,4.12,5.62,0.04

1023,-4.46,-10.41,-14.45,4.36,3.83,-0.05

1023,-3.46,-10.69,-12.88,3.58,3.78,0.11

1023,-2.93,-11.17,-11.27,3.66,3.85,-0.06

1023,-3.28,-11.45,-10.72,3.85,4.43,-0.01

1023,-3.62,-11.35,-10.68,3.83,4.95,0.02

1023,-3.58,-11.29,-10.54,4.14,4.44,-0.07

1023,-2.96,-11.01,-10.88,3.78,4.21,0.08

1023,-2.48,-10.61,-11.13,3.94,3.72,0.24

1023,-2.47,-10.03,-11.37,4.23,3.47,0.31

1023,-2.8

1023,-9.69,-2.45,-2.88,1.86,2.47,0.19

1023,-9.59,-2.51,-2.94,1.90,2.48,0.18

1023,-9.55,-2.54,-2.96,1.88,2.49,0.18

1023,-9.61,-2.48,-2.91,1.87,2.47,0.19

1023,-9.74,-2.40,-2.83,1.86,2.44,0.19

1023,-9.78,-2.38,-2.80,1.84,2.44,0.20

1023,-9.75,-2.40,-2.83,1.85,2.43,0.19

1023,-9.64,-2.48,-2.90,1.87,2.49,0.19

1023,-9.58,-2.51,-2.95,1.88,2.47,0.18

1023,-9.57,-2.51,-2.96,1.88,2.48,0.18

1023,-9.64,-2.47,-2.88,1.87,2.47,0.20

1023,-9.72,-2.39,-2.83,1.85,2.44,0.20

1023,-9.78,-2.36,-2.81,1.84,2.43,0.19

1023,-9.72,-2.41,-2.83,1.86,2.45,0.19

1023,-9.64,-2.50,-2.90,1.86,2.47,0.19

1023,-9.57,-2.52,-2.95,1.89,2.49,0.18

1023,-9.60,-2.52,-2.94,1.88,2.48,0.18

1023,-9.66,-2.44,-2.88,1.87,2.46,0.19

1023,-9.74,-2.40,-2.83,1.85,2.45,0.20

1023,-9.76,-2.37,-2.82,1.84,2.43,0.20

1023,-9.73,-2.41,-2.86,1.86,2.45,0.20

1023,-9.63,-2.49,-2.92,1.87,2.48,0.19

1023,-9.58,-2.51,-2.95,1.89,2.49,0.19

1023,-9.60,-2.51,-2.93,1.87,2.48,0.19

1023,-9.68,-2.44,-2.87,1.87,2.46,0.18

1023,-9.75,-2.38,-2.83,1.

0,-9.69,-2.44,-2.90,1.87,2.47,0.18

2,-9.70,-2.44,-2.87,1.87,2.47,0.19

1,-9.70,-2.45,-2.88,1.87,2.46,0.19

0,-9.70,-2.42,-2.88,1.87,2.47,0.19

0,-9.69,-2.45,-2.88,1.87,2.46,0.19

1,-9.68,-2.45,-2.91,1.88,2.46,0.19

0,-9.67,-2.45,-2.90,1.87,2.46,0.19

1,-9.69,-2.43,-2.90,1.88,2.46,0.18

1,-9.68,-2.44,-2.88,1.86,2.46,0.19

0,-9.69,-2.44,-2.88,1.87,2.47,0.19

0,-9.70,-2.44,-2.87,1.87,2.46,0.19

0,-9.69,-2.44,-2.89,1.86,2.47,0.19

0,-9.68,-2.44,-2.89,1.87,2.45,0.18

0,-9.66,-2.47,-2.90,1.87,2.48,0.19

0,-9.68,-2.45,-2.90,1.87,2.47,0.19

1,-9.71,-2.43,-2.87,1.88,2.46,0.18

0,-9.70,-2.43,-2.88,1.87,2.46,0.19

1,-9.69,-2.45,-2.88,1.87,2.46,0.19

1,-9.67,-2.46,-2.90,1.87,2.47,0.19

2,-9.68,-2.46,-2.90,1.88,2.48,0.19

0,-9.68,-2.46,-2.89,1.87,2.47,0.18

0,-9.68,-2.44,-2.89,1.88,2.46,0.19

1,-9.71,-2.44,-2.87,1.87,2.46,0.19

1,-9.70,-2.43,-2.88,1.87,2.48,0.19

0,-9.68,-2.44,-2.88,1.88,2.46,0.19

0,-9.68,-2.44,-2.90,1.88,2.49,0.19

0,-9.69,-2.45,-2.89,1.87,2.48,0.18

0,-9.69,-2.45,-2.89,1.88,2.4

1,-9.71,-2.44,-2.90,1.89,2.47,0.18

0,-9.70,-2.45,-2.89,1.88,2.47,0.19

0,-9.70,-2.43,-2.89,1.87,2.48,0.18

2,-9.68,-2.44,-2.88,1.88,2.48,0.18

2,-9.70,-2.45,-2.88,1.87,2.45,0.18

0,-9.70,-2.44,-2.88,1.88,2.47,0.20

0,-9.71,-2.45,-2.89,1.89,2.46,0.18

1,-9.70,-2.43,-2.87,1.88,2.47,0.18

0,-9.69,-2.44,-2.88,1.88,2.47,0.18

2,-9.71,-2.44,-2.89,1.87,2.47,0.19

0,-9.69,-2.44,-2.88,1.87,2.48,0.19

0,-9.71,-2.44,-2.88,1.87,2.45,0.18

0,-9.68,-2.45,-2.89,1.86,2.48,0.19

0,-9.71,-2.44,-2.90,1.88,2.47,0.19

0,-9.70,-2.43,-2.89,1.87,2.47,0.18

0,-9.69,-2.43,-2.89,1.89,2.46,0.18

0,-9.70,-2.44,-2.90,1.87,2.47,0.19

1,-9.69,-2.44,-2.89,1.87,2.47,0.19

0,-9.69,-2.44,-2.89,1.88,2.48,0.18

0,-9.69,-2.45,-2.89,1.87,2.47,0.18

1,-9.70,-2.43,-2.89,1.87,2.46,0.20

3,-9.71,-2.44,-2.89,1.88,2.47,0.18

1,-9.69,-2.43,-2.90,1.89,2.47,0.18

0,-9.69,-2.44,-2.88,1.87,2.47,0.19

0,-9.69,-2.45,-2.89,1.87,2.46,0.19

0,-9.70,-2.43,-2.89,1.88,2.47,0.19

0,-9.71,-2.44,-2.90,1.88,2.47,0.18

0,-9.69,-2.44,-2.87,1.88,2.4

# Logging D, Force Data


In [23]:
import numpy as np 
import matplotlib.pyplot as plt
import pyrealsense2 as rs
import cv2
import serial


# Setup:
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)


# Create an object to read  
# from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera 
# is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 
    
# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 



# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
mag_deci = 4
decimation = rs.decimation_filter(mag_deci)
threshold = rs.threshold_filter(0.15, 0.6)

n_trial = 24
depthwriter = cv2.VideoWriter('fcss_leo_depth_test'+str(n_trial)+'.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         30, (int(frame_width/mag_deci), int(frame_height/mag_deci)), 1) 

print("type anything to start")
start_test = input()

addr = "COM4" ## serial port to read data from
baud = 115200 ## baud rate for instrument

ser = serial.Serial(
    port = addr,\
    baudrate = baud)

fcss_filename = "fcss_leo_load_test"+str(n_trial)+".csv"
datafile=open(fcss_filename, 'a')

colorizer = rs.colorizer()
colorizer.set_option(rs.option.color_scheme, 3); 

# Start streaming
pipe.start(cfg)

try: 
    while True:
        
        try:
            ser_bytes = ser.readline()
            decoded_bytes = (ser_bytes[0:len(ser_bytes)-2].decode("utf-8").split(","))



            write_line = ",".join(decoded_bytes) + str("\n")
            print(write_line)
            datafile.write(write_line)

            # Wait for a coherent pair of frames: depth and color
            frames = pipe.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            
            if not depth_frame:
                continue
            
            # Decimation 
            depth_frame = decimation.process(depth_frame)

            # Thresholding
            depth_frame = threshold.process(depth_frame)
        
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = np.asanyarray(colorizer.colorize(depth_frame).get_data())
        
            
            
            depth_save = cv2.resize(depth_colormap, (int(frame_width/mag_deci), int(frame_height/mag_deci)))
            
            
            depthwriter.write(depth_save)
            cv2.imshow('Depth', depth_save)

            key = cv2.waitKey(1)

            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                video.release() 
                depthwriter.release()
                break 
                
        except:
            pass
            break
            
        

finally:
    ser.close()
    datafile.close()
    pipe.stop()
    cv2.destroyAllWindows()
    video.release() 
    depthwriter.release()

type anything to start
a
1023,-11.75,-1.62,-2.54,2.06,2.39,0.06

1023,-11.76,-1.61,-2.55,2.04,2.40,0.08

1023,-11.75,-1.63,-2.54,2.04,2.41,0.09

1023,-11.75,-1.63,-2.55,2.08,2.38,0.07

1023,-11.76,-1.63,-2.55,2.05,2.39,0.08

1023,-11.75,-1.62,-2.54,2.04,2.41,0.08

1023,-11.75,-1.62,-2.56,2.05,2.40,0.08

1023,-11.76,-1.63,-2.55,2.06,2.39,0.08

1023,-11.75,-1.62,-2.54,2.05,2.41,0.08

1023,-11.76,-1.62,-2.54,2.04,2.40,0.08

1023,-11.75,-1.63,-2.53,2.05,2.40,0.08

1023,-11.75,-1.64,-2.55,2.05,2.40,0.08

1023,-11.75,-1.62,-2.53,2.05,2.40,0.08

1023,-11.77,-1.63,-2.55,2.05,2.40,0.08

1023,-11.75,-1.62,-2.53,2.05,2.40,0.08

1023,-11.75,-1.62,-2.55,2.05,2.40,0.08

1023,-11.76,-1.63,-2.55,2.04,2.40,0.08

1023,-11.77,-1.62,-2.54,2.05,2.40,0.08

1023,-11.74,-1.62,-2.55,2.06,2.39,0.08

1023,-11.75,-1.62,-2.55,2.05,2.41,0.08

1023,-11.75,-1.63,-2.55,2.05,2.40,0.08

1023,-11.76,-1.62,-2.54,2.06,2.39,0.09

1023,-11.75,-1.63,-2.55,2.06,2.39,0.07

1023,-11.76,-1.64,-2.53,2.05,2.40,0.09

1023,-11.76,-1.

1023,-11.76,-1.63,-2.54,2.06,2.41,0.08

1023,-11.75,-1.63,-2.54,2.05,2.39,0.08

1023,-11.75,-1.62,-2.55,2.06,2.40,0.08

1023,-11.74,-1.63,-2.54,2.05,2.39,0.07

1023,-11.76,-1.62,-2.54,2.04,2.41,0.09

1023,-11.76,-1.61,-2.54,2.05,2.40,0.07

1023,-11.75,-1.63,-2.55,2.06,2.40,0.09

1023,-11.76,-1.62,-2.56,2.05,2.42,0.08

1023,-11.76,-1.62,-2.54,2.06,2.41,0.08

1023,-11.75,-1.63,-2.54,2.07,2.38,0.08

1023,-11.75,-1.61,-2.55,2.06,2.40,0.07

1023,-11.75,-1.63,-2.55,2.05,2.41,0.09

1023,-11.75,-1.61,-2.55,2.05,2.41,0.07

1023,-11.75,-1.62,-2.53,2.06,2.40,0.09

1023,-11.74,-1.62,-2.54,2.06,2.39,0.07

1023,-11.75,-1.62,-2.55,2.06,2.40,0.08

1023,-11.76,-1.63,-2.55,2.05,2.39,0.08

1023,-11.75,-1.63,-2.54,2.06,2.41,0.07

1023,-11.76,-1.63,-2.55,2.06,2.40,0.09

1023,-11.75,-1.63,-2.55,2.05,2.39,0.07

1023,-11.75,-1.61,-2.54,2.06,2.40,0.08

1023,-11.75,-1.63,-2.53,2.06,2.39,0.07

1023,-11.75,-1.62,-2.54,2.06,2.40,0.08

1023,-11.76,-1.61,-2.54,2.05,2.41,0.08

1023,-11.75,-1.63,-2.53,2.06,2.40,0.07



1023,-11.75,-1.63,-2.54,2.06,2.41,0.08

1023,-11.75,-1.62,-2.54,2.06,2.39,0.08

1023,-11.75,-1.63,-2.55,2.05,2.41,0.09

1023,-11.77,-1.63,-2.55,2.05,2.40,0.08

1023,-11.76,-1.61,-2.54,2.05,2.39,0.08

1023,-11.75,-1.63,-2.54,2.05,2.40,0.07

1023,-11.75,-1.62,-2.53,2.05,2.41,0.08

1023,-11.75,-1.61,-2.54,2.06,2.40,0.08

1023,-11.75,-1.61,-2.55,2.06,2.39,0.08

1023,-11.76,-1.62,-2.55,2.04,2.40,0.08

1023,-11.76,-1.61,-2.54,2.04,2.40,0.08

1023,-11.76,-1.62,-2.55,2.06,2.40,0.08

1023,-11.75,-1.63,-2.53,2.05,2.40,0.08

1023,-11.76,-1.63,-2.55,2.05,2.41,0.09

1023,-11.75,-1.63,-2.55,2.06,2.40,0.08

1023,-11.76,-1.62,-2.53,2.06,2.39,0.08

1023,-11.75,-1.61,-2.56,2.05,2.40,0.08

1023,-11.77,-1.63,-2.55,2.04,2.41,0.08

1023,-11.76,-1.63,-2.55,2.05,2.40,0.07

1023,-11.76,-1.63,-2.55,2.05,2.40,0.08

1023,-11.75,-1.63,-2.55,2.04,2.40,0.08

1023,-11.75,-1.62,-2.55,2.04,2.40,0.08

1023,-11.76,-1.63,-2.54,2.06,2.40,0.08

1023,-11.76,-1.63,-2.55,2.05,2.40,0.08

1023,-11.75,-1.63,-2.54,2.05,2.40,0.08



1023,-49.46,-5.54,-22.08,8.13,15.79,-0.37

1023,-49.11,-4.77,-22.44,8.50,15.02,-0.69

1023,-49.68,-4.54,-22.97,8.58,14.66,-0.87

1023,-49.87,-4.54,-23.27,8.54,14.36,-1.06

1023,-50.49,-4.44,-24.75,8.76,14.55,-1.83

1023,-51.48,-3.75,-26.64,9.42,14.92,-2.08

1023,-51.73,-2.79,-27.96,10.04,15.26,-3.69

1023,-51.29,-2.80,-28.21,10.07,15.78,-4.30

1023,-51.26,-3.35,-27.92,9.84,16.39,-4.65

1023,-51.10,-3.58,-27.00,10.13,16.08,-4.81

1023,-50.84,-4.49,-25.48,10.12,15.72,-4.12

1023,-50.57,-5.98,-23.85,9.91,15.30,-3.55

1023,-50.24,-8.01,-22.19,9.37,14.44,-2.39

1023,-50.36,-9.88,-20.89,9.24,13.82,-1.77

1023,-49.76,-12.14,-19.65,9.57,13.45,-1.26

1023,-48.30,-14.55,-18.22,9.42,12.92,-0.67

1023,-48.77,-15.84,-16.25,9.54,12.27,-0.43

1023,-50.17,-17.17,-14.08,10.07,11.30,-0.81

1023,-49.51,-18.98,-12.66,10.36,10.72,-0.10

1023,-49.65,-20.67,-11.40,10.99,10.59,0.64

1023,-50.61,-21.77,-9.74,11.33,10.50,1.79

1023,-50.87,-22.75,-8.24,11.66,9.91,2.05

1023,-51.37,-23.46,-6.35,11.93,9.86,3.88

1

1023,-56.07,-13.00,-10.97,10.06,11.57,0.20

1023,-56.72,-12.63,-10.67,10.14,11.70,0.20

1023,-57.61,-12.20,-10.33,10.22,11.85,0.20

1023,-58.36,-11.80,-9.97,10.34,11.89,0.27

1023,-59.06,-11.34,-9.52,10.50,11.94,0.38

1023,-59.90,-10.99,-9.07,10.61,12.00,0.46

1023,-60.54,-10.85,-8.75,10.61,12.00,0.58

1023,-60.91,-10.65,-8.39,10.55,12.00,0.73

1023,-61.23,-10.47,-8.09,10.54,12.00,0.80

1023,-61.55,-10.30,-7.79,10.53,12.02,0.85

1023,-62.19,-10.08,-7.51,10.56,12.10,0.88

1023,-62.91,-9.66,-7.32,10.65,12.23,0.86

1023,-63.50,-9.18,-7.21,10.77,12.27,0.78

1023,-63.86,-8.82,-7.16,10.85,12.26,0.71

1023,-63.87,-8.61,-7.23,10.91,12.23,0.58

1023,-63.85,-8.54,-7.29,10.90,12.34,0.49

1023,-63.91,-8.45,-7.37,10.92,12.36,0.46

1023,-64.00,-8.39,-7.32,10.93,12.33,0.49

1023,-64.02,-8.47,-7.15,10.87,12.34,0.59

1023,-63.96,-8.69,-7.18,10.78,12.37,0.67

1023,-63.71,-8.90,-7.38,10.74,12.31,0.73

1023,-63.36,-8.93,-7.57,10.76,12.20,0.74

1023,-63.10,-8.99,-7.71,10.78,12.16,0.68

1023,-62.86,-9.09,-7

...

1023,-53.67,-23.68,-3.87,11.61,9.21,1.12

1023,-54.10,-23.71,-3.22,11.81,9.10,1.48

1023,-54.68,-23.67,-2.58,12.13,8.95,1.72

1023,-55.23,-23.59,-2.13,12.33,8.82,1.83

1023,-55.50,-23.39,-1.87,12.50,8.60,1.75

1023,-55.79,-23.21,-1.66,12.62,8.44,1.61

1023,-55.92,-23.24,-1.38,12.54,8.48,1.52

1023,-56.36,-23.23,-1.17,12.53,8.47,1.44

1023,-56.81,-23.04,-0.93,12.57,8.43,1.48

1023,-57.11,-22.81,-0.63,12.58,8.45,1.53

1023,-57.62,-22.55,-0.45,12.70,8.47,1.54

1023,-58.17,-22.17,-0.21,12.82,8.46,1.53

1023,-58.37,-22.00,-0.03,12.72,8.55,1.48

1023,-58.69,-21.82,0.06,12.61,8.63,1.49

1023,-59.15,-21.44,0.18,12.67,8.66,1.57

1023,-59.55,-21.07,0.30,12.70,8.70,1.64

1023,-60.08,-20.71,0.46,12.73,8.76,1.69

1023,-60.47,-20.42,0.56,12.71,8.78,1.72

1023,-60.76,-20.08,0.54,12.72,8.76,1.70

1023,-61.04,-19.66,0.46,12.77,8.72,1.65

1023,-61.45,-19.21,0.39,12.78,8.73,1.61

1023,-61.71,-18.76,0.37,12.75,8.75,1.48

1023,-62.02,-18.38,0.31,12.70,8.89,1.40

1023,-62.46,-17.95,0.25,12.68,9.07,1.38

102

1023,-53.67,-16.13,-11.00,10.39,11.33,0.33

1023,-53.64,-16.70,-10.45,10.40,11.19,0.44

1023,-53.84,-17.20,-9.95,10.48,11.01,0.58

1023,-54.14,-17.70,-9.33,10.64,10.87,0.74

1023,-54.20,-18.11,-8.73,10.75,10.78,0.85

1023,-54.31,-18.40,-8.26,10.88,10.68,0.94

1023,-54.65,-18.69,-7.84,11.07,10.62,1.04

1023,-54.95,-18.95,-7.42,11.20,10.50,1.12

1023,-54.99,-19.21,-7.01,11.28,10.33,1.12

1023,-54.87,-19.48,-6.69,11.23,10.20,1.15

1023,-54.85,-19.69,-6.48,11.22,10.04,1.19

1023,-54.92,-19.84,-6.20,11.25,9.93,1.26

1023,-55.11,-19.95,-5.88,11.32,9.85,1.30

1023,-55.35,-19.99,-5.52,11.35,9.82,1.34

1023,-55.67,-19.97,-5.23,11.45,9.79,1.38

1023,-56.00,-19.95,-5.00,11.54,9.77,1.41

1023,-56.22,-19.86,-4.80,11.58,9.78,1.40

1023,-56.51,-19.71,-4.59,11.63,9.79,1.36

1023,-56.99,-19.57,-4.38,11.73,9.81,1.36

1023,-57.27,-19.45,-4.24,11.73,9.82,1.34

1023,-57.40,-19.29,-4.12,11.72,9.78,1.33

1023,-57.52,-19.18,-4.04,11.72,9.72,1.31

1023,-57.77,-19.00,-3.96,11.73,9.69,1.30

1023,-57.99,-18.81,-3

1023,-62.66,-7.94,-7.57,10.47,9.30,0.69

1023,-63.40,-7.48,-8.07,9.86,10.22,0.22

1023,-63.89,-6.64,-9.08,9.46,10.98,-0.10

1023,-64.02,-5.59,-10.17,9.31,12.08,-0.41

1023,-64.42,-4.54,-11.43,9.16,13.23,-0.69

1023,-64.14,-3.68,-12.65,8.92,14.09,-0.87

1023,-63.40,-2.89,-14.03,8.64,14.96,-0.73

1023,-63.29,-1.89,-15.16,8.27,15.61,-0.68

1023,-63.56,-1.12,-15.69,7.65,16.22,-0.24

1023,-63.75,-0.82,-16.60,7.16,16.64,0.12

1023,-63.34,-0.56,-16.89,6.99,17.21,0.41

1023,-63.68,-0.46,-16.94,6.68,18.00,0.80

1023,-63.62,-0.34,-16.92,6.49,18.38,0.84

1023,-63.44,-0.32,-16.97,6.17,18.74,0.70

1023,-62.91,-0.48,-17.43,5.69,18.91,0.58

1023,-62.68,-0.53,-17.89,5.41,18.88,0.43

1023,-61.72,-0.86,-18.62,5.19,18.49,0.36

1023,-60.08,-1.55,-19.21,5.22,18.00,0.28

1023,-59.81,-1.82,-20.31,5.46,17.28,0.11

1023,-59.52,-0.61,-21.69,5.75,16.15,0.22

1023,-59.90,1.09,-22.47,5.96,15.19,0.67

1023,-60.71,3.08,-23.27,6.27,15.24,0.71

1023,-61.46,4.38,-23.38,6.62,15.28,0.75

1023,-61.69,5.01,-23.52,6.54,15.9

1023,-11.85,-1.54,-2.60,2.03,2.46,0.07

1023,-11.83,-1.53,-2.59,2.02,2.47,0.07

1023,-11.86,-1.53,-2.59,2.03,2.46,0.07

1023,-11.85,-1.53,-2.59,2.03,2.45,0.08

1023,-11.83,-1.55,-2.60,2.03,2.46,0.07

1023,-11.85,-1.53,-2.61,2.02,2.46,0.08

1023,-11.84,-1.52,-2.60,2.02,2.45,0.07

1023,-11.84,-1.54,-2.60,2.03,2.45,0.07

1023,-11.84,-1.53,-2.58,2.03,2.45,0.08

1023,-11.85,-1.52,-2.60,2.02,2.46,0.08

1023,-11.84,-1.53,-2.58,2.03,2.44,0.08

1023,-11.85,-1.53,-2.60,2.03,2.46,0.07

1023,-11.84,-1.54,-2.60,2.02,2.45,0.08

1023,-11.85,-1.52,-2.59,2.03,2.47,0.07

1023,-11.85,-1.54,-2.58,2.03,2.46,0.07

1023,-11.84,-1.54,-2.60,2.03,2.46,0.07

1023,-11.84,-1.53,-2.59,2.03,2.47,0.07

1023,-11.84,-1.55,-2.59,2.03,2.44,0.08

1023,-11.84,-1.53,-2.59,2.03,2.45,0.07

1023,-11.84,-1.53,-2.59,2.03,2.47,0.08

1023,-11.84,-1.53,-2.59,2.02,2.45,0.07

1023,-11.83,-1.52,-2.58,2.03,2.45,0.08

1023,-11.85,-1.54,-2.59,2.04,2.45,0.07

1023,-11.83,-1.53,-2.59,2.04,2.45,0.07

1023,-11.87,-1.53,-2.58,2.02,2.45,0.08



0,-11.85,-1.53,-2.58,2.04,2.46,0.07

1,-11.84,-1.54,-2.60,2.04,2.46,0.07

0,-11.84,-1.54,-2.58,2.03,2.46,0.07

0,-11.84,-1.53,-2.59,2.05,2.46,0.07

0,-11.83,-1.54,-2.59,2.03,2.46,0.07

0,-11.85,-1.54,-2.58,2.03,2.46,0.07

0,-11.85,-1.53,-2.58,2.04,2.46,0.07

0,-11.84,-1.53,-2.58,2.04,2.45,0.07

0,-11.85,-1.53,-2.59,2.03,2.46,0.07

0,-11.85,-1.53,-2.59,2.03,2.46,0.08

0,-11.84,-1.55,-2.58,2.04,2.45,0.06

0,-11.85,-1.54,-2.58,2.04,2.46,0.07

0,-11.85,-1.54,-2.59,2.02,2.48,0.07

0,-11.84,-1.54,-2.59,2.03,2.46,0.07

0,-11.85,-1.54,-2.57,2.05,2.44,0.07

0,-11.84,-1.54,-2.59,2.04,2.45,0.06

0,-11.85,-1.54,-2.59,2.03,2.45,0.08

0,-11.84,-1.54,-2.58,2.03,2.45,0.07

0,-11.85,-1.54,-2.58,2.04,2.46,0.07

2,-11.85,-1.55,-2.58,2.03,2.45,0.07

1,-11.85,-1.54,-2.60,2.04,2.46,0.07

0,-11.84,-1.53,-2.59,2.05,2.45,0.07

0,-11.84,-1.53,-2.59,2.04,2.45,0.07

1,-11.85,-1.54,-2.58,2.03,2.47,0.07

0,-11.85,-1.55,-2.60,2.03,2.46,0.07

0,-11.85,-1.54,-2.58,2.05,2.44,0.07

0,-11.84,-1.54,-2.59,2.03,2.45,0.07

0

In [20]:
depthwriter.release()